In [1]:
!pip install -q \
langchain \
langgraph \
faiss-cpu \
pymupdf \
pillow \
pytesseract \
opencv-python \
streamlit \
numpy


In [2]:
!apt-get update
!apt-get install -y tesseract-ocr


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 10s (397 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-

In [3]:
!tesseract --version


tesseract 4.1.1
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8


In [4]:
import os

base_dirs = [
    "data/documents",
    "data/images",
    "src/loaders",
    "src/embeddings",
    "src/graph",
    "src/qa"
]

for d in base_dirs:
    os.makedirs(d, exist_ok=True)

print("Project folder structure created successfully")


Project folder structure created successfully


In [5]:
from google.colab import files

print("Upload PDFs and Images")
uploaded = files.upload()


Upload PDFs and Images


Saving chart for multi model.png to chart for multi model.png
Saving masif cv - Copy.pdf to masif cv - Copy.pdf


In [6]:
import shutil

for file in uploaded.keys():
    if file.lower().endswith(".pdf"):
        shutil.move(file, f"data/documents/{file}")
    else:
        shutil.move(file, f"data/images/{file}")

print("Files moved to respective folders")


Files moved to respective folders


In [7]:
print("Documents:", os.listdir("data/documents"))
print("Images:", os.listdir("data/images"))


Documents: ['masif cv - Copy.pdf']
Images: ['chart for multi model.png']


In [8]:
import fitz  # PyMuPDF
import os

PDF_DIR = "data/documents"

def extract_text_from_pdfs(pdf_dir):
    documents = []

    for file in os.listdir(pdf_dir):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_dir, file)
            doc = fitz.open(pdf_path)

            full_text = ""
            for page in doc:
                full_text += page.get_text()

            documents.append({
                "source": file,
                "type": "pdf",
                "content": full_text
            })

    return documents


pdf_texts = extract_text_from_pdfs(PDF_DIR)

print(f"Extracted text from {len(pdf_texts)} PDF(s)")


Extracted text from 1 PDF(s)


In [9]:
pdf_texts[0]["content"][:500]


' \nMOHD MASIF AHMED \nEmail: csmasifahmed@gmail.com | Phone: +91 7893894313 | LinkedIn  \n \nPROFESSIONAL SUMMARY \nAspiring Data Scientist / Machine Learning Engineer with hands-on experience in Python, data \nanalysis, machine learning, NLP, and automation. Seeking an entry-level role to apply analytical and \nproblem-solving skills to real-world problems. \n \nEDUCATION \nB.E. Computer Science (Data Science) — Nawab Shah Alam Khan College of Engineering and \nTechnology \n2021–2025 | CGPA: 6.95/10 \n \nHig'

In [10]:
import pytesseract
from PIL import Image

IMAGE_DIR = "data/images"

def extract_text_from_images(image_dir):
    image_docs = []

    for file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, file)

        try:
            img = Image.open(img_path)
            text = pytesseract.image_to_string(img)

            image_docs.append({
                "source": file,
                "type": "image",
                "content": text
            })
        except Exception as e:
            print(f"Error processing {file}: {e}")

    return image_docs


image_texts = extract_text_from_images(IMAGE_DIR)

print(f"Extracted text from {len(image_texts)} image(s)")


Extracted text from 1 image(s)


In [11]:
image_texts[0]["content"][:500]


'6 Recruiter actions on your profile\n\n \n\n(al Actions ©) (Naukri profile viewed (1) ) (Resume downloaded (1) ) (NVite sent (5) )\n\n \n\nF G\nFoundeverl5 HR recruiter\nHR Lead at Foundever AVP TA at Genpact\nMUMBAI GURGAON\n\nNot specified Not specified\n\x0c'

In [12]:
all_documents = pdf_texts + image_texts

print(f"Total multimodal documents: {len(all_documents)}")


Total multimodal documents: 2


In [13]:
{
  "source": "file_name",
  "type": "pdf / image",
  "content": "extracted text"
}


{'source': 'file_name', 'type': 'pdf / image', 'content': 'extracted text'}

In [14]:
!pip install -q sentence-transformers


In [15]:
from langchain_core.documents import Document

documents = []

for doc in all_documents:
    documents.append(
        Document(
            page_content=doc["content"],
            metadata={
                "source": doc["source"],
                "type": doc["type"]
            }
        )
    )

print(f"Prepared {len(documents)} documents for embedding")


Prepared 2 documents for embedding


In [16]:
!pip install -q langchain-community sentence-transformers


In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("Embeddings loaded successfully")



/tmp/ipython-input-904552999.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings loaded successfully


In [18]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents,
    embedding_model
)

print("FAISS vector store created successfully")


FAISS vector store created successfully


In [19]:
query = "What information is shown in the chart or image?"

results = vectorstore.similarity_search(query, k=3)

for i, res in enumerate(results):
    print(f"\nResult {i+1}")
    print("Source:", res.metadata)
    print(res.page_content[:300])



Result 1
Source: {'source': 'chart for multi model.png', 'type': 'image'}
6 Recruiter actions on your profile

 

(al Actions ©) (Naukri profile viewed (1) ) (Resume downloaded (1) ) (NVite sent (5) )

 

F G
Foundeverl5 HR recruiter
HR Lead at Foundever AVP TA at Genpact
MUMBAI GURGAON

Not specified Not specified


Result 2
Source: {'source': 'masif cv - Copy.pdf', 'type': 'pdf'}
 
MOHD MASIF AHMED 
Email: csmasifahmed@gmail.com | Phone: +91 7893894313 | LinkedIn  
 
PROFESSIONAL SUMMARY 
Aspiring Data Scientist / Machine Learning Engineer with hands-on experience in Python, data 
analysis, machine learning, NLP, and automation. Seeking an entry-level role to apply analytica


In [20]:
!pip install -q transformers accelerate


In [24]:
!pip install -q langchain


In [28]:
# 1. Retrieve relevant documents manually
query = "Explain the chart and summarize the document"

docs = vectorstore.similarity_search(query, k=4)

context = "\n\n".join([d.page_content for d in docs])

# 2. Load LLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 3. Ask question with retrieved context
prompt = f"""
Use the context below to answer the question.

Context:
{context}

Question:
{query}

Answer:
"""

answer = llm.invoke(prompt)
print(answer)


Device set to use cpu


6 Recruiter actions on your profile (al Actions ) (Naukri profile viewed (1) ) (Resume downloaded (1) ) (NVite sent (5) ) F G Foundeverl5 HR recruiter HR Lead at Foundever AVP TA at Genpact MUMBAI GURGAON Not specified Not specified MOHD MASIF AHMED Email: csmasifahmed@gmail.com | Phone: +91 7893894313 | LinkedIn PROFESSIONAL SUMMARY Aspiring Data Scientist / Machine Learning Engineer with hands-on experience in Python, data analysis, machine learning, NLP, and automation Seeking an entry-level role to apply analytical and problem-solving skills to real-world problems.
